# Kак роботы планируют траекторию движения

In [3]:
# Не изменяйте эту ячейку и запустите её перед началом работы!
!pip install pygame
import math

You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


![1](./Image/1.png)

В этом ноутбуке мы рассмотрим проблему нахождения кратчайшего пути на карте. Эта задача используется в робототехнике, компьютерных играх и сервисах такси, доставки и т.п. В данном ноутбуке карта будет представлять из себя сетку, в которой одни клетки проходимы, а другие представляют собой препятствия. Существуют разные спрособы определять соседей клетки, например, соседями могут являться только смежные клетки с данной или можно еще разрешать ходить по диагонали при некоторых ограничениях. В этом задании будем предполагать, что мы можем ходить только по смежным клеткам: вверх, вниз, влево и вправо.

Базовым алгоритмом поиска кратчайшего пути является алгоритм Дийкстры. Суть алгоритма проста — обходить вершины в порядке возрастания расстояния от начальной вершины. Для этого мы храним для каждой вершины метку $g$ — минимальное известное расстояние от начальной вершины до этой (изначально для стартовой вершины оно равно 0, а для всех остальных $\infty$).
 На каждом шаге мы рассматриваем вершину с минимальным g среди нерассмотренных (Понятно, что на первой итерации выбрана будет стартовая вершина). Нетрудно доказать, что при неотрицательных весах при рассмотрении $g$ этой вершины равно истинному значению минимального расстояния.  Далее, на текущей итерации, из рассматриваемой вершины производятся релаксации: мы пытаемся улучшить значение g для всех соседей этой вершины, то есть
 
$$neighbour.g = min(neighbour.g,\  self.g+dist(self, neighbour)).$$

![2](./Image/dijkstra.gif)

В нашем случае мы решаем задачу поиска кратчайшего пути до одной конкретной вершины, значит во время рассмотрения этой самой вершины алгоритм можно и нужно остановить. Помимо этого в нашей ситуации много упрощений, поскольку граф клетчатый, соседи - это несколько смежных клеток, а веса рёбер равны единице.

Однако алгоритм Дийкстры имеет большой минус: если мы ищем кратчайший путь от начальной вершины до конкретной, то мы рассмотрим ВСЕ вершины, расстояние до которых меньше чем до целевой, даже если они в противоположной от неё стороне. Например, если мы хотим долететь из Новосибирска до Владивостока, то зачем лететь с пересадкой в Москве, ведь она находится в противоположной стороне. Чтобы это исправить, рассмотрим $h$ — эвристическую функцию вершины. Это оценка расстояния от неё до конечной. И тогда логично брать вершину не с минимальным $g$, а минимальным значением $f = h + g$. Такой алогоритм называется A* (A-star, А-звёздочка). Вот наглядный пример его работы:

In [4]:
# Может стоит добавить две гифки - сравнение дийкстры и a*?

![3](./Image/astar.gif)

Как видите, алгоритм не рассматривает многие вершины, которые алгоритм Дийсткры бы рассмотрел. Таким образом, хорошая метрика для A* как раз поможет избежать лишних действий

Введём класс Node, описывающий вершину. У него есть следующие поля:
- i, j - координаты вершины
- g
- f = g + h
- parent - вершина, из которой мы пришли в данную. По этому полю мы сможем восстановить путь.

In [5]:
EPS = 1e-9

In [6]:
class Node:
    def __init__(self, i, j, g = math.inf, h = math.inf, parent = None):
        self.i = i
        self.j = j
        self.g = g
        self.f = self.g + h       
        self.parent = parent
    
    def __eq__(self, other):
        return (self.i == other.i) and (self.j == other.j)
    
    def __gt__(self, other):
        return self.f - EPS > other.f
    
    def __hash__(self):
        return 31 + 7 * (hash(self.i) + 7 * hash(self.j))
    
    def __str__(self):
        return 'i: {0}, j: {1}, g: {2}, F: {3}, parent: {4}'.format(self.i, self.j, self.g, self.f, self.parent)

In [7]:
# TODO: add Grid implementation

### Реализация Open и Closed

В нашей реализации мы будем использовать два класса — Open и Closed. Open содержит нерассмотренные вершины, а Closed — уже рассмотренные. В качестве примера приведены реализации на List-е. Однако, такая реализация крайне не эффективная. Напишите свою реализацию, использующую  более эффективную структуру данных

In [8]:
class OpenBase:
    def __init__(self):
        pass
    
    def __len__(self):
        pass
    
    def __iter__(self):
        pass

    # isEmpty должна сообщать о том, пустое множество или нет
    def isEmpty(self):
        pass

    # AddNode это метод добавляющий или обновляющий вершину в OPEN
    # Не забудьте обработать следующие случаи:
    # - вершина уже в OPEN но новое g-value лучше;
    # - вершина уже в OPEN но новое g-value хуже;
    # - вершина не в OPEN.
    def AddNode(self, item: Node):
        pass

    # GetBestNode это метод, возвращающий лучшую вершину из OPEN (с наименьшим f-value) и удаляющий ее из OPEN
    def GetBestNode(self):
        pass

In [9]:
class OpenList(OpenBase):
    def __init__(self):
        self.elements = []
    
    def __iter__(self):
        return iter(self.elements)

    def __len__(self):
        return len(self.elements)

    def isEmpty(self):
        if len(self.elements) != 0:
            return False
        return True

    def GetBestNode(self):
        bestF = math.inf
        bestCoord = 0
        for i in range(len(self.elements)):
            if (self.elements[i].f < bestF) or (abs(self.elements[i].f - bestF) < EPS):
                bestCoord = i
                bestF = self.elements[i].f
                
        best = self.elements.pop(bestCoord)
        return best
    

    def AddNode(self, item):
        for coord in range(len(self.elements)):
            if self.elements[coord].i == item.i and self.elements[coord].j == item.j:
                if (self.elements[coord].g > item.g) or (abs(self.elements[coord].g - item.g) < EPS):
                    self.elements[coord].f = item.f
                    self.elements[coord].g = item.g
                    self.elements[coord].parent = item.parent
                    return
                else:
                    return
        self.elements.append(item)
        return

In [10]:
import heapq

class YourOpen(OpenBase):
    def __init__(self):
        self.elements = []
        self.in_open = set()
    
    def __iter__(self):
        return iter(sorted(list([n for n in self.elements if n in self.in_open])))

    def __len__(self):
        return len(self.in_open)

    def isEmpty(self):
        return len(self.elements) == 0
    
    def GetBestNode(self):
        min_node = heapq.heappop(self.elements)
        self.in_open.remove(min_node)
        while len(self.elements) != 0 and self.elements[0] not in self.in_open:
            heapq.heappop(self.elements)
        return min_node

    def AddNode(self, item: Node):
        self.in_open.add(item)
        heapq.heappush(self.elements, item)

Посмотрите на импелементацию Open выше и убедитесь, что вы всё понимаете. Как видите, от Open требуется выполнять следующие операции: добавлять вершину (или обновлять значение её g) и быстро доставать ноду с минимальным f-value. Подумайте, какая структура данных могут эффективно выполнять эти операции? (Подсказка: в стандартной библиотеке Python есть подходящая)

In [11]:
class ClosedBase:

    def __init__(self):
        pass

    def __iter__(self):
        pass
    
    def __len__(self):
        pass
    
    
    def AddNode(self, item: Node, *args):
        pass

    def WasExpanded(self, item: Node, *args):
        pass

In [12]:
class ClosedList (ClosedBase):
    
    def __init__(self):
        self.elements = []

    def __iter__(self):
        return iter(self.elements)
    
    def __len__(self):
        return len(self.elements)
    
    # AddNode is the method that inserts the node to CLOSED
    def AddNode(self, item, *args):
        self.elements.append(item)

    # WasExpanded is the method that checks if a node has been expanded
    def WasExpanded(self, item, *args):
        return item in self.elements

In [13]:
class YourClosed (ClosedBase):
    
    def __init__(self):
        self.elements = set()

    def __iter__(self):
        return iter(self.elements)
    
    def __len__(self):
        return len(self.elements)
    
    # AddNode is the method that inserts the node to CLOSED
    def AddNode(self, item : Node, *args):
        self.elements.add(item)

    # WasExpanded is the method that checks if a node has been expanded
    def WasExpanded(self, item : Node, *args):
        return item in self.elements

Как видно, от Closed требуется выполнять операции добавления и узнавать, есть ли элемент в множестве. Уверены, что вы знакомы с нужной струкурой данных.

## Алгоритм A*

В качестве эвристической функции будем использовать Манхэтэнское расстояние — сумму расстояний по вертикали и горизонтали.

In [14]:
def ManhattanMetric(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

А теперь вам предстоит реализовать используя сам алгоритм. !!!TODO написать про его парамерты

In [15]:
def A_star(grid, start, end, OPEN, CLOSED, h):
    open_set = OPEN
    closed_set = CLOSED
    start_node = Node(start[0], start[1], 0, h(start[0], start[1], end[0], end[1]))
    end_node = Node(end[0], end[1])
    open_set.AddNode(start_node)

    found_flag = False
    last_v = None
    while not open_set.isEmpty():
        v = open_set.GetBestNode()
        closed_set.AddNode(v)
        if v == end_node:
            found_flag = True
            last_v = v
            break

        g_val = v.g
        for to_i, to_j in grid.GetNeighbors(v.i, v.j):
            if not closed_set.WasExpanded(Node(to_i, to_j)):
                h_curr = h(to_i, to_j, end_node.i, end_node.j)
                new_node = Node(to_i, to_j, g_val + 1, h_curr, parent=v)
                open_set.AddNode(new_node)
    return found_flag, last_v

In [16]:
from game import start_game

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


Left click - закрасить клетку. Первая закрашенная клетка - начало. Вторая - конец. Дальше - стены. Right click - удалить клетки. R - очистить доску. Space - запустить A*

In [17]:
start_game(A_star, OpenList, YourClosed, ManhattanMetric)

In [16]:
# TODO сделать тесты